In [ ]:
from alpha_vantage.timeseries import TimeSeries
from pprint import pprint
import pandas as pd
import time


alpha_vantage
https://alpha-vantage.readthedocs.io/en/latest/
Python module to get stock data from the Alpha Vantage API Alpha Vantage http://www.alphavantage.co/ is a free JSON APIs for stock market data, plus a comprehensive set of technical indicators. This project is a python wrapper around this API to offer python plus json/pandas support. I hope you enjoy it. It requires a free API, that can be requested on http://www.alphavantage.co/support/#api-key.

The code is hosted in github: https://github.com/RomelTorres/alpha_vantage 

the specific module: https://github.com/RomelTorres/alpha_vantage/blob/develop/alpha_vantage/timeseries.py  is well documented.   

Note: Rather than the entire series of prices, the more efficent approach would be to fetch single values, using  def get_batch_stock_quotes(self, symbols): or def get_quote_endpoint(self, symbol): functions. hwr, the fucntionaly does not currently work for non-US shares.  So fetching a larger set anf triming to the last value is what is done below.  

Set some initial and trial values 

In [2]:
#Alphavantage_API_key = 'YOUR_API_KEY'
Alphavantage_API_key = '21GBFSMHLICNID9Y'

# symbols = ['HWDN.l','C'] # test data, keep for troubleshooting for problmes with share_data file

datafile='share_data.csv'  # where the exising data is kept
earningsfile = 'epsdiv.csv' #where Share Symbol, eps and div data is kept. Updated for each share after the company reports. (ca once per quarter)

pause_call = 21 #time to wait between API calls a multiple of this is use on error to allow time for server to allo more calls 


# Define functions

In [3]:
def getnewdata(symbol, days=1, apikey=Alphavantage_API_key):
    ts = TimeSeries(key=apikey, output_format='pandas')
    df, meta_df = ts.get_daily(symbol=symbol, outputsize='compact')
    #df, meta_df = ts.get_batch_stock_quotes(symbols=shrsymbol)  # only works with US stocks
    data = df.tail(days)
    return data

Keyword Arguments:
    days: use in dataframe tail function, the number of data points 
    symbol:  the symbol for the equity we want to get its data
    outputsize:  The size of the call, supported values are 'compact' and 'full; the first returns the last 100 points in the data series, and 'full' returns the full-length intraday times series, commonly above 1MB (default 'compact')

In [4]:
def getexistingdata(datafile):
    with open(datafile, 'r') as f:
        data_existing = pd.read_csv(f, index_col='date') # set index to date as Alpha Vantage uses date as index
        return data_existing

# Main

Get new data using list, symbols

In [5]:
#Retreive the existing data sorred on disk in preperation for merging with the new data.
data_existing = getexistingdata(datafile)
#pprint(data_existing)

KeyError: 'date'

In [ ]:
# data_point = pd.DataFrame()  # creating an empty dataframe into which the new data will be placed 

Data file:  'epsdiv.cdv'  Share symbols, eps and div data is held in a csv file.  This file can be edditied when the eps and div values change or when a new symbol is added or deleted. 

In [ ]:
#Read the symbols for each share to be used in the call to AV
print('reading share Symbols to retreive from file', earningsfile)
data = pd.read_csv(earningsfile,index_col=False) # creat a dataframe from the  csv file 
symbols = data['Symbol'].drop_duplicates().values.tolist()  # create a list of only the unique values in 'Symbol' column
print('read ', len(symbols),' unique share symbols')

In [ ]:
#Read the eps and div values for each share in to a dict 
print('reading share eps and div values for the Symbols from file', earningsfile )
epsdiv=pd.read_csv(earningsfile, header=0, index_col=0).to_dict() #create a dict of the Symbols and eps, div values to be put in the new data frame with values retreived from AV
#todo could the csv file be read in once to a dataframe an exported to the Symbol list and eps/div dict from the singel DF

In [ ]:
cnt = 1 # for a counter to track progress of the retreival process. 

In [ ]:
print('retreiving  data')
for s in symbols:                    #iterate over the Symbols retreived from earningsfile
    print(cnt,' of', len(symbols))
    cnt =cnt+1
    
    try:
        d = getnewdata(s)       # retreives dataframe of date and prices 
        d['eps'] = epsdiv['eps'][s]   #create a column 'eps' with the value from the dict epsdiv
        d['PE']  = d['4. close']/epsdiv['eps'][s] #calc the PE and add to dataframe
        d['div'] = epsdiv['div'][s]
        d['yld'] = epsdiv['div'][s]/d['4. close']*100 #calc the yld and add to dataframe
        d['Symbol']= str(s)
    except Exception as E:
        cnt_dwn = pause_call*3  #Pause the API calls if the rate was too high and tossed an error
        print('error encounted while retreiving data ', s)
        print(E)
        print('retreval will pause for ', cnt_dwn,  'seconds' )
        while cnt_dwn >0:
            print(cnt_dwn)
            time.sleep(1)
            cnt_dwn = cnt_dwn -1 
            
    d.rename(columns={'1. open':'open','2. high':'high','3. low':'low','4. close':'close','5. volume':'volume'}, inplace=True)
    #print('New data', s)
    pprint(d)
    print('')
 
    data_existing = pd.concat([data_existing,d])
    # print('data_existing.tail()',data_existing.tail())
    #print('')
    
    time.sleep(pause_call)  #Time between API calls, the AV server limits the number of calls in a given time 
    

the 'Time Series (Daily) call fails at 25 calls with 15 sec intervals.  try greater time

In [ ]:
data_existing.drop_duplicates(keep='last', inplace=True)   # remove duplicate values, keeping the later. This allows earnings data to be updated in later runs
print('data_existing after drop dups')
#print(data_existing.tail(len(symbols)))
#pprint(data_existing)
print('')

Check the data looks correct

In [ ]:
with open('share_data.csv', 'a') as f:
    data_existing.to_csv(f, header=False)